# M4: CD Pipeline & Deployment

**Objective:** Implement Continuous Deployment of the containerized model to a target environment.

**Tasks:**
1. Deployment Target (Kubernetes/Docker Compose)
2. CD/GitOps Flow
3. Smoke Tests/Health Checks

---

## 1. Setup

In [1]:
import sys
import os
import subprocess

sys.path.append(os.path.abspath('..'))
print("✓ Setup complete!")

✓ Setup complete!


## 2. Deployment Option 1: Kubernetes

In [2]:
# Display Kubernetes deployment manifest
with open('../deployment/kubernetes/deployment.yaml', 'r') as f:
    k8s_manifest = f.read()

print("Kubernetes Deployment Manifest:")
print("=" * 60)
print(k8s_manifest[:1500], "\n... (truncated) ...")
print(f"\nTotal lines: {len(k8s_manifest.splitlines())}")

Kubernetes Deployment Manifest:
apiVersion: apps/v1
kind: Deployment
metadata:
  name: cats-dogs-classifier
  labels:
    app: cats-dogs-classifier
    version: v1
spec:
  replicas: 2
  selector:
    matchLabels:
      app: cats-dogs-classifier
  template:
    metadata:
      labels:
        app: cats-dogs-classifier
        version: v1
    spec:
      containers:
      - name: classifier
        image: YOUR_DOCKERHUB_USERNAME/cats-dogs-classifier:latest
        imagePullPolicy: Always
        ports:
        - containerPort: 8000
          name: http
          protocol: TCP
        env:
        - name: PYTHONUNBUFFERED
          value: "1"
        resources:
          requests:
            memory: "512Mi"
            cpu: "500m"
          limits:
            memory: "1Gi"
            cpu: "1000m"
        livenessProbe:
          httpGet:
            path: /health
            port: 8000
          initialDelaySeconds: 30
          periodSeconds: 10
          timeoutSeconds: 5
          f

In [3]:
print("Kubernetes Deployment Components:")
print("=" * 60)
print("\n1. Deployment")
print("   - Replicas: 2")
print("   - Image: YOUR_DOCKERHUB_USERNAME/cats-dogs-classifier:latest")
print("   - Resources: 500m-1000m CPU, 512Mi-1Gi memory")
print("   - Liveness probe: /health every 10s")
print("   - Readiness probe: /health every 5s")
print("\n2. Service")
print("   - Type: LoadBalancer")
print("   - Port: 80 → 8000")
print("\n3. HorizontalPodAutoscaler")
print("   - Min replicas: 2")
print("   - Max replicas: 5")
print("   - CPU target: 70%")
print("   - Memory target: 80%")

Kubernetes Deployment Components:

1. Deployment
   - Replicas: 2
   - Image: YOUR_DOCKERHUB_USERNAME/cats-dogs-classifier:latest
   - Resources: 500m-1000m CPU, 512Mi-1Gi memory
   - Liveness probe: /health every 10s
   - Readiness probe: /health every 5s

2. Service
   - Type: LoadBalancer
   - Port: 80 → 8000

3. HorizontalPodAutoscaler
   - Min replicas: 2
   - Max replicas: 5
   - CPU target: 70%
   - Memory target: 80%


In [4]:
print("Deploy to Kubernetes:")
print("=" * 60)
print("\n# Prerequisites")
print("# Install minikube or kind for local cluster")
print("minikube start")
print("\n# Update image in deployment.yaml first!")
print("# Replace YOUR_DOCKERHUB_USERNAME with your username")
print("\n# Apply manifests")
print("kubectl apply -f deployment/kubernetes/deployment.yaml")
print("\n# Check deployment")
print("kubectl get deployments")
print("kubectl get pods")
print("kubectl get svc")
print("\n# Port forward to access")
print("kubectl port-forward svc/cats-dogs-classifier-service 8000:80")
print("\n# Test")
print("curl http://localhost:8000/health")

Deploy to Kubernetes:

# Prerequisites
# Install minikube or kind for local cluster
minikube start

# Update image in deployment.yaml first!
# Replace YOUR_DOCKERHUB_USERNAME with your username

# Apply manifests
kubectl apply -f deployment/kubernetes/deployment.yaml

# Check deployment
kubectl get deployments
kubectl get pods
kubectl get svc

# Port forward to access
kubectl port-forward svc/cats-dogs-classifier-service 8000:80

# Test
curl http://localhost:8000/health


## 3. Deployment Option 2: Docker Compose

In [5]:
# Display Docker Compose file
with open('../deployment/docker-compose/docker-compose.yml', 'r') as f:
    compose_content = f.read()

print("Docker Compose Configuration:")
print("=" * 60)
print(compose_content)

Docker Compose Configuration:
version: '3.8'

services:
  classifier:
    build:
      context: .
      dockerfile: Dockerfile
    image: cats-dogs-classifier:latest
    container_name: cats-dogs-classifier
    ports:
      - "8000:8000"
    environment:
      - PYTHONUNBUFFERED=1
    volumes:
      - ./models:/app/models:ro
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:8000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s
    networks:
      - mlops-network
    deploy:
      resources:
        limits:
          cpus: '1'
          memory: 1G
        reservations:
          cpus: '0.5'
          memory: 512M

  # Optional: MLflow tracking server
  mlflow:
    image: python:3.10-slim
    container_name: mlflow-server
    ports:
      - "5000:5000"
    command: >
      sh -c "pip install mlflow &&
             mlflow server 
             --host 0.0.0.0 
             --port 5000 
             --back

In [6]:
print("Docker Compose Services:")
print("=" * 60)
print("\n1. classifier (port 8000)")
print("   - Main API service")
print("   - Resource limits: 1 CPU, 1G memory")
print("   - Health checks enabled")
print("\n2. mlflow (port 5000)")
print("   - Experiment tracking server")
print("   - SQLite backend")
print("   - Persistent storage")
print("\n3. prometheus (port 9090)")
print("   - Monitoring and metrics")
print("   - Scrapes classifier metrics")
print("   - Persistent storage")

Docker Compose Services:

1. classifier (port 8000)
   - Main API service
   - Resource limits: 1 CPU, 1G memory
   - Health checks enabled

2. mlflow (port 5000)
   - Experiment tracking server
   - SQLite backend
   - Persistent storage

3. prometheus (port 9090)
   - Monitoring and metrics
   - Scrapes classifier metrics
   - Persistent storage


In [7]:
print("Deploy with Docker Compose:")
print("=" * 60)
print("\n# Navigate to compose directory")
print("cd deployment/docker-compose")
print("\n# Start all services")
print("docker-compose up -d")
print("\n# Check status")
print("docker-compose ps")
print("\n# View logs")
print("docker-compose logs -f classifier")
print("\n# Access services:")
print("  - API: http://localhost:8000")
print("  - MLflow: http://localhost:5000")
print("  - Prometheus: http://localhost:9090")
print("\n# Stop all services")
print("docker-compose down")

Deploy with Docker Compose:

# Navigate to compose directory
cd deployment/docker-compose

# Start all services
docker-compose up -d

# Check status
docker-compose ps

# View logs
docker-compose logs -f classifier

# Access services:
  - API: http://localhost:8000
  - MLflow: http://localhost:5000
  - Prometheus: http://localhost:9090

# Stop all services
docker-compose down


## 4. Smoke Tests

In [8]:
# Display smoke test script
with open('../scripts/smoke_test.sh', 'r') as f:
    smoke_test = f.read()

print("Smoke Test Script:")
print("=" * 60)
print(smoke_test[:1000], "\n... (truncated) ...")
print(f"\nTotal lines: {len(smoke_test.splitlines())}")

Smoke Test Script:
#!/bin/bash
# Smoke tests for deployed service

set -e

# Configuration
SERVICE_URL="${SERVICE_URL:-http://localhost:8000}"
MAX_RETRIES=30
RETRY_INTERVAL=2

echo "=== Running Smoke Tests ==="
echo "Service URL: $SERVICE_URL"
echo ""

# Function to wait for service
wait_for_service() {
    echo "Waiting for service to be ready..."
    for i in $(seq 1 $MAX_RETRIES); do
        if curl -f -s "${SERVICE_URL}/health" > /dev/null 2>&1; then
            echo "Service is ready!"
            return 0
        fi
        echo "Attempt $i/$MAX_RETRIES: Service not ready yet, waiting..."
        sleep $RETRY_INTERVAL
    done
    echo "Service failed to become ready after $MAX_RETRIES attempts"
    return 1
}

# Test 1: Health Check
test_health_check() {
    echo "Test 1: Health Check"
    response=$(curl -s -w "\n%{http_code}" "${SERVICE_URL}/health")
    http_code=$(echo "$response" | tail -n1)
    body=$(echo "$response" | head -n-1)
    
    if [ "$http_code" -eq 200 ] || [ 

In [9]:
print("Smoke Tests:")
print("=" * 60)
print("\nThe smoke_test.sh script performs:")
print("  1. Health check endpoint test")
print("  2. Root endpoint test")
print("  3. Model info endpoint test")
print("  4. Prediction endpoint test (with dummy image)")
print("  5. Metrics endpoint test")
print("\nRun smoke tests:")
print("bash scripts/smoke_test.sh")
print("\nWith custom URL:")
print("SERVICE_URL=http://your-service:8000 bash scripts/smoke_test.sh")
print("\nExpected output:")
print("  ✓ All critical tests passed!")

Smoke Tests:

The smoke_test.sh script performs:
  1. Health check endpoint test
  2. Root endpoint test
  3. Model info endpoint test
  4. Prediction endpoint test (with dummy image)
  5. Metrics endpoint test

Run smoke tests:
bash scripts/smoke_test.sh

With custom URL:
SERVICE_URL=http://your-service:8000 bash scripts/smoke_test.sh

Expected output:
  ✓ All critical tests passed!


## 5. CD Pipeline Integration

In [10]:
print("CD Pipeline Flow (GitHub Actions):")
print("=" * 60)
print("\n1. Trigger: Push to main branch")
print("   └─ Test job passes")
print("   └─ Build job completes")
print("\n2. Deploy job (optional, commented in workflow)")
print("   ├─ Checkout code")
print("   ├─ Setup kubectl")
print("   ├─ Configure kubeconfig")
print("   ├─ Apply Kubernetes manifests")
print("   ├─ Wait for rollout")
print("   └─ Run smoke tests")
print("\n3. If smoke tests fail:")
print("   └─ Pipeline fails")
print("   └─ Rollback (if configured)")
print("\n4. If smoke tests pass:")
print("   └─ Deployment complete")
print("   └─ Service available")

CD Pipeline Flow (GitHub Actions):

1. Trigger: Push to main branch
   └─ Test job passes
   └─ Build job completes

2. Deploy job (optional, commented in workflow)
   ├─ Checkout code
   ├─ Setup kubectl
   ├─ Configure kubeconfig
   ├─ Apply Kubernetes manifests
   ├─ Wait for rollout
   └─ Run smoke tests

3. If smoke tests fail:
   └─ Pipeline fails
   └─ Rollback (if configured)

4. If smoke tests pass:
   └─ Deployment complete
   └─ Service available


## 6. Monitor Deployment

In [11]:
print("Monitoring Deployed Service:")
print("=" * 60)
print("\nKubernetes:")
print("  # Pod status")
print("  kubectl get pods -w")
print("  \n  # Pod logs")
print("  kubectl logs -f <pod-name>")
print("  \n  # Resource usage")
print("  kubectl top pods")
print("  \n  # HPA status")
print("  kubectl get hpa")
print("\nDocker Compose:")
print("  # Service status")
print("  docker-compose ps")
print("  \n  # Service logs")
print("  docker-compose logs -f classifier")
print("  \n  # Resource usage")
print("  docker stats")
print("\nApplication Metrics:")
print("  # Prometheus (if running)")
print("  http://localhost:9090")
print("  \n  # MLflow (if running)")
print("  http://localhost:5000")

Monitoring Deployed Service:

Kubernetes:
  # Pod status
  kubectl get pods -w
  
  # Pod logs
  kubectl logs -f <pod-name>
  
  # Resource usage
  kubectl top pods
  
  # HPA status
  kubectl get hpa

Docker Compose:
  # Service status
  docker-compose ps
  
  # Service logs
  docker-compose logs -f classifier
  
  # Resource usage
  docker stats

Application Metrics:
  # Prometheus (if running)
  http://localhost:9090
  
  # MLflow (if running)
  http://localhost:5000


## Summary

### ✓ Completed Tasks:

1. **Deployment Target**
   - Kubernetes manifests (Deployment, Service, HPA)
   - Docker Compose configuration
   - Resource limits and requests defined
   - Health checks configured

2. **CD/GitOps Flow**
   - GitHub Actions deploy job
   - Automatic deployment on main push
   - Image pull from registry
   - Rolling updates configured

3. **Smoke Tests/Health Checks**
   - Comprehensive smoke test script
   - Tests all critical endpoints
   - Fails pipeline if tests fail
   - Post-deployment validation

### Deployment Options:
- **Kubernetes**: Production-grade, scalable
- **Docker Compose**: Quick local/dev deployment

### Next Steps:
- Choose deployment method
- Deploy to target environment
- Run smoke tests
- Proceed to M5 for monitoring